In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasetsforecast.m4 import M4
from config_models import MODEL_LIST

from neuralforecast.losses.numpy import mae, mse, smape, mqloss

In [ ]:
source_dataset = 'M4'
#target_dataset = 'M3'
k_shot=0
experiment_id = '20230813'

#final_df = pd.DataFrame(columns=['model', 'smape', 'mae', 'mqloss'])
mae_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
smape_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)
mqloss_df = pd.DataFrame(columns = ['dataset', 'frequency'] + MODEL_LIST)

for target_dataset in ['M3']: # , 'AirPassengers', 'ILI', 'TrafficL'
    for frequency in ['yearly','quarterly','monthly','daily']:
        mae_list = []
        smape_list = []
        mqloss_list = []
        for model in MODEL_LIST:
            results_df = pd.read_csv(f'./results/transferability/forecasts/{target_dataset}/{frequency}/{model}_{k_shot}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)
            #print(results_df.shape)
            #results_df = results_df.groupby('unique_id').head(12)
            #print(results_df.shape)

            smape_loss = 100*smape(results_df['y'], results_df.iloc[:,8])
            mae_loss = mae(results_df['y'], results_df.iloc[:,8])
            mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,4:13], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
            mae_list.append(np.round(mae_loss,3))
            smape_list.append(np.round(smape_loss,3))
            mqloss_list.append(np.round(mq_loss,3))

        mae_df = pd.concat([mae_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mae_list))})], ignore_index=True)
        smape_df = pd.concat([smape_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, smape_list))})], ignore_index=True)
        mqloss_df = pd.concat([mqloss_df, pd.DataFrame({'dataset': [target_dataset], 'frequency': [frequency], **dict(zip(MODEL_LIST, mqloss_list))})], ignore_index=True)

In [ ]:
mae_df

In [ ]:
# for i in range(1):
#     unique_id = results_df['unique_id'].unique()[np.random.randint(1)]
#     plot_df = results_df.query('unique_id == @unique_id')

#     plt.figure(figsize=(20, 5))
#     plt.plot(plot_df['y'], label='y')
#     plt.plot(plot_df[model_name], label='y_hat')
#     plt.show()

In [ ]:
import pandas as pd
import numpy as npx

from datasetsforecast.m4 import M4
from datasetsforecast.m3 import M3
from datasetsforecast.long_horizon import LongHorizon
from datasetsforecast.losses import mae
from statsforecast.utils import AirPassengersDF

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS, Naive

In [ ]:
Y_df_target, *_ = M3.load(directory='./', group='Monthly')
Y_df_target['ds'] = pd.to_datetime(Y_df_target['ds'])
frequency = 'M'
test_size = 18

In [ ]:
unique_ids = Y_df_target['unique_id'].unique()
unique_ids = np.random.choice(unique_ids,size=20)
Y_df_target = Y_df_target[Y_df_target['unique_id'].isin(unique_ids)].reset_index(drop=True)
Y_df_target = Y_df_target.groupby('unique_id').tail(200).reset_index(drop=True)

In [ ]:
Y_df_target

In [ ]:
season_length = 12 # Monthly data 
#season_length = 24 # Monthly data 

# Include the models you imported
models = [
    AutoARIMA(season_length=season_length),
    #AutoETS(season_length=season_length),
    Naive()
]

# Instansiate the StatsForecast class as sf
sf = StatsForecast(
    df=Y_df_target,
    models=models,
    freq='M', 
    n_jobs=-1
)

crossvalidation_df = sf.cross_validation(
    df = Y_df_target,
    h = horizon,
    step_size = 1,
    n_windows = 1
  )

In [ ]:
crossvalidation_df = crossvalidation_df.reset_index(drop=True)
crossvalidation_df

In [ ]:
for model_name in ['AutoARIMA', 'Naive']:
    print('Model:', model_name)
    mae_loss = mae(crossvalidation_df['y'], crossvalidation_df[model_name])
    print('mae:', mae_loss)